## M4 yearly 02 to 20 model

In [1]:
# import datetime
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datlib.FRED import *
from datlib.plots import *
import pandas_datareader.data as web

%matplotlib inline

# Import Statsmodels

from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [2]:
#FRED.py
#. . . 
def bil_to_mil(series):
    return series* 10**3
# . . .
#fedProject.py
# . . .
data_codes  = {# Assets
               "Balance Sheet: Total Assets ($ Mil)": "WALCL",
               "Balance Sheet Securities, Prem-Disc, Repos, and Loans ($ Mil)": "WSRLL",
               "Balance Sheet: Securities Held Outright ($ Mil)": "WSHOSHO",
               ### breakdown of securities holdings ###
               "Balance Sheet: U.S. Treasuries Held Outright ($ Mil)":"WSHOTSL",
               "Balance Sheet: Federal Agency Debt Securities ($ Mil)" : "WSHOFADSL",
               "Balance Sheet: Mortgage-Backed Securities ($ Mil)": "WSHOMCB",
               # other forms of lending
               "Balance Sheet: Repos ($ Mil)": "WORAL",
               "Balance Sheet: Central Bank Liquidity Swaps ($ Mil)" : "SWPT",
               "Balance Sheet: Direct Lending ($ Mil)" : "WLCFLL",
               # unamortized value of securities held (due to changes in interest rates)
               "Balance Sheet: Unamortized Security Premiums ($ Mil)": "WUPSHO",
               # Liabilities
               "Balance Sheet: Total Liabilities ($ Mil)" : "WLTLECL",
               "Balance Sheet: Federal Reserve Notes Outstanding ($ Mil)" : "WLFN",
               "Balance Sheet: Reverse Repos ($ Mil)": "WLRRAL",
               ### Major share of deposits 
               "Balance Sheet: Deposits from Dep. Institutions ($ Mil)":"WLODLL",
               "Balance Sheet: U.S. Treasury General Account ($ Mil)": "WDTGAL",
               "Balance Sheet: Other Deposits ($ Mil)": "WOTHLB",
               "Balance Sheet: All Deposits ($ Mil)": "WLDLCL",
               # Capital
               "Balance Sheet: Total Capital": "WCTCL",
               # Interest Rates
               "Unemployment Rate": "UNRATE",
               "Nominal GDP ($ Bil)":"GDP",
               "Real GDP ($ Bil)":"GDPC1",
               "GDP Deflator":"GDPDEF",
               "CPI":"CPIAUCSL",
               "Core PCE":"PCEPILFE",
               "Private Investment":"GPDI",
               "Base: Total ($ Mil)": "BOGMBASE",
               "Base: Currency in Circulation ($ Bil)": "WCURCIR",
               "1 Month Treasury Rate (%)": "DGS1MO",
               "3 Month Treasury Rate (%)": "DGS3MO",               
               "1 Year Treasury Rate (%)": "DGS1",
               "2 Year Treasury Rate (%)": "DGS2",
               "10 Year Treasury Rate (%)": "DGS10",
               "30 Year Treasury Rate (%)": "DGS30",               
               "Effective Federal Funds Rate (%)": "DFF",
               "Federal Funds Target Rate (Pre-crisis)":"DFEDTAR",
               "Federal Funds Upper Target":"DFEDTARU",
               "Federal Funds Lower Target":"DFEDTARL",
               "Interest on Reserves (%)": "IOER",
               "VIX": "VIXCLS",
                "5 Year Forward Rate": "T5YIFR"
               }

inflation_target = 2

unemployment_target = 4
# Select start and end dates
start = datetime.datetime(2000, 1, 1)
end = datetime.datetime.today()

## year variable automatically adjusts the numper of periods  
#   per year in light of data frequency
annual_div = {"Q":4,
             "W":52,
             "M":12}
### choose frequency
freq = "M"
### set periods per year
year = annual_div[freq]


In [42]:
#data cleaning, importing

d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')
df = pd.read_csv('M4-17.csv', parse_dates=['Date'], date_parser=d_parser)
df

,Date,Effective Federal Funds Rate (%),Log TA,Log CC,Log M4,Log NGDP
0,2002-12-31,1.238387,27.310541,27.240157,27.486147,30.231068
1,2003-01-31,1.235161,27.309049,27.244493,27.492783,30.044622
2,2003-02-28,1.262143,27.304357,27.245112,27.497724,30.231068
3,2003-03-31,1.252903,27.307575,27.250365,27.501099,30.231068
4,2003-04-30,1.258000,27.325754,27.255681,27.504974,30.056953
...,...,...,...,...,...,...
212,2020-08-31,0.095161,29.573481,28.326427,28.379096,30.549194
213,2020-09-30,0.090000,29.583818,28.337206,28.383158,30.549194
214,2020-10-31,0.090000,29.596371,28.344070,28.382883,30.698032
215,2020-11-30,0.086333,29.604840,28.351605,28.385972,30.549194


In [43]:
df['Date_at_year_month'] = df['Date'].dt.strftime('%Y-%m')

In [44]:
column_names = {'Date_at_year_month':'DATE',
                'Log M4':'M4',
                'Log TA': 'TA',
                'Log CC':'CC',
                'Effective Federal Funds Rate (%)':'FFR',
               'Log NGDP': 'NGDP'}

# rename columns
df = df.rename(columns = column_names)

df

,Date,FFR,TA,CC,M4,NGDP,DATE
0,2002-12-31,1.238387,27.310541,27.240157,27.486147,30.231068,2002-12
1,2003-01-31,1.235161,27.309049,27.244493,27.492783,30.044622,2003-01
2,2003-02-28,1.262143,27.304357,27.245112,27.497724,30.231068,2003-02
3,2003-03-31,1.252903,27.307575,27.250365,27.501099,30.231068,2003-03
4,2003-04-30,1.258000,27.325754,27.255681,27.504974,30.056953,2003-04
...,...,...,...,...,...,...,...
212,2020-08-31,0.095161,29.573481,28.326427,28.379096,30.549194,2020-08
213,2020-09-30,0.090000,29.583818,28.337206,28.383158,30.549194,2020-09
214,2020-10-31,0.090000,29.596371,28.344070,28.382883,30.698032,2020-10
215,2020-11-30,0.086333,29.604840,28.351605,28.385972,30.549194,2020-11


In [45]:
df = df.set_index('DATE')
df = df.drop(['Date'], axis = 1)
df

,FFR,TA,CC,M4,NGDP
DATE,,,,,
2002-12,1.238387,27.310541,27.240157,27.486147,30.231068
2003-01,1.235161,27.309049,27.244493,27.492783,30.044622
2003-02,1.262143,27.304357,27.245112,27.497724,30.231068
2003-03,1.252903,27.307575,27.250365,27.501099,30.231068
2003-04,1.258000,27.325754,27.255681,27.504974,30.056953
...,...,...,...,...,...
2020-08,0.095161,29.573481,28.326427,28.379096,30.549194
2020-09,0.090000,29.583818,28.337206,28.383158,30.549194
2020-10,0.090000,29.596371,28.344070,28.382883,30.698032


In [8]:
#data = df

In [46]:
#1st diff
df = df.diff(year).dropna()
df

,FFR,TA,CC,M4,NGDP
DATE,,,,,
2003-12,-0.254194,0.045793,0.057056,0.038431,0.000000
2004-01,-0.238065,0.047454,0.049309,0.036962,0.064906
2004-02,-0.254901,0.043829,0.047986,0.038326,0.000000
2004-03,-0.251290,0.043121,0.043676,0.044078,0.000000
2004-04,-0.254000,0.033697,0.042352,0.046991,0.068332
...,...,...,...,...,...
2020-08,-2.030645,0.614565,0.136296,0.258132,0.000000
2020-09,-1.953000,0.615588,0.139360,0.255551,0.000000
2020-10,-1.739677,0.586545,0.142360,0.246285,-0.010046


In [47]:
from statsmodels.tsa.stattools import kpss

def kpss_test(df):    
    statistic, p_value, n_lags, critical_values = kpss(df.values)
    
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
        
print('KPSS Test: M4')
kpss_test(df['M4'])
print('KPSS Test: FFR')
kpss_test(df['FFR'])
print('KPSS Test: TA')
kpss_test(df['TA'])
print('KPSS Test: CC')
kpss_test(df['CC'])
print('KPSS Test: NGDP')
kpss_test(df['NGDP'])

KPSS Test: M4
KPSS Statistic: 0.3732689149151507
p-value: 0.0886771918469178
num lags: 9
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: FFR
KPSS Statistic: 0.13419203408957553
p-value: 0.1
num lags: 9
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: TA
KPSS Statistic: 0.12287240967760166
p-value: 0.1
num lags: 9
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: CC
KPSS Statistic: 0.7226659744043655
p-value: 0.011484911417784951
num lags: 9
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: NGDP
KPSS Statistic: 0.16946285224929708
p-value: 0.1
num lags: 9
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739


In [40]:
#2nd diff
df = df.diff(year).dropna()
df

,FFR,TA,CC,M4,NGDP
DATE,,,,,
2004-12,1.426129,0.016904,-0.006413,0.031544,0.000000
2005-01,1.520323,0.012750,0.000492,0.025943,0.003474
2005-02,1.749446,0.016558,0.002776,0.021509,0.000000
2005-03,1.878710,0.017146,0.008396,0.009490,0.000000
2005-04,2.035000,0.019289,0.007823,0.004141,-0.003614
...,...,...,...,...,...
2020-08,-2.241613,0.730777,0.090972,0.205752,0.000000
2020-09,-2.041333,0.714671,0.094606,0.198904,0.000000
2020-10,-1.381613,0.635169,0.096478,0.183344,-0.051510


In [33]:
data = data.diff(year).dropna()

In [7]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR
import statsmodels.api as sm
import copy
import pingouin
from scipy.stats import pearsonr
from datlib.ts_tests import *
from statsmodels.tsa.adfvalues import *
import warnings
warnings.simplefilter("ignore")
import statsmodels
statsmodels.__file__

'C:\\Users\\HP\\anaconda3\\lib\\site-packages\\statsmodels\\__init__.py'

In [41]:
from statsmodels.tsa.stattools import kpss

def kpss_test(df):    
    statistic, p_value, n_lags, critical_values = kpss(df.values)
    
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
        
print('KPSS Test: M4')
kpss_test(df['M4'])
print('KPSS Test: FFR')
kpss_test(df['FFR'])
print('KPSS Test: TA')
kpss_test(df['TA'])
print('KPSS Test: CC')
kpss_test(df['CC'])
print('KPSS Test: NGDP')
kpss_test(df['NGDP'])

KPSS Test: M4
KPSS Statistic: 0.3363375412790879
p-value: 0.1
num lags: 9
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: FFR
KPSS Statistic: 0.10893335700218604
p-value: 0.1
num lags: 9
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: TA
KPSS Statistic: 0.08757128639907757
p-value: 0.1
num lags: 8
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: CC
KPSS Statistic: 0.08748231460487188
p-value: 0.1
num lags: 9
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: NGDP
KPSS Statistic: 0.0413508128442893
p-value: 0.1
num lags: 8
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739


In [35]:
##Testing Causation using Granger’s Causality Test

from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data_new, variables, test='ssr_chi2test', verbose=False):
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data_new[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(df, variables = df.columns) 

,FFR_x,TA_x,CC_x,M4_x,NGDP_x
FFR_y,1.0000,0.0000,0.0000,0.0853,0.1430
TA_y,0.0000,1.0000,0.0027,0.0000,0.3830
CC_y,0.0000,0.0000,1.0000,0.0000,0.0581
M4_y,0.0000,0.0002,0.0001,1.0000,0.0001
NGDP_y,0.0002,0.0000,0.0000,0.0000,1.0000


In [42]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import statsmodels.api as sm
def joh_output(res):
    output = pd.DataFrame([res.lr2,res.lr1],
                           index=['max_eig_stat',"trace_stat"])
    print(output.T,'\n')
    print("Critical values(90%, 95%, 99%) of max_eig_stat\n",res.cvm,'\n')
    print("Critical values(90%, 95%, 99%) of trace_stat\n",res.cvt,'\n')
res = coint_johansen(df, 0 ,1)
joh_output(res)

   max_eig_stat  trace_stat
0     63.777276  149.862510
1     35.928298   86.085233
2     31.584378   50.156936
3     12.825332   18.572558
4      5.747226    5.747226 

Critical values(90%, 95%, 99%) of max_eig_stat
 [[31.2379 33.8777 39.3693]
 [25.1236 27.5858 32.7172]
 [18.8928 21.1314 25.865 ]
 [12.2971 14.2639 18.52  ]
 [ 2.7055  3.8415  6.6349]] 

Critical values(90%, 95%, 99%) of trace_stat
 [[65.8202 69.8189 77.8202]
 [44.4929 47.8545 54.6815]
 [27.0669 29.7961 35.4628]
 [13.4294 15.4943 19.9349]
 [ 2.7055  3.8415  6.6349]] 



The null hypothesis was that the time series are not cointegrated.

The trace statistics tell us whether the sum of the eigenvalues is 0. The null hypothesis, r<=0 gives us a trace statistic of 17.895, hence the null hypothesis can be rejected at a 95% confidence level, as the magnitude of the trace statistic is greater than the critical value, note that the Johansen test only gives the magnitude of the output, hence we need not worry about the signs.

The eigen statistics stores the eigenvalues in decreasing order of magnitude, they tell us how strongly cointegrated the series are or how strong is the tendency to mean revert. In our example, the eigen statistic for the null hypothesis can be rejected at a 95% confidence level, because 17.5694 is greater than 14.2639.

The eigenvectors give us the equation of the mean-reverting linear combination of the time series. The eigenvector corresponding to the highest eigenvalue represents the portfolio which has the greatest mean-reverting property. The null hypothesis was that the time series are not cointegrated, hence when we reject the null hypothesis and accept the alternate hypothesis, we suggest that the series are cointegrated.

In [33]:
from statsmodels.tsa.api import VAR

model = VAR(df)
for i in [1,2,3,4,5,6,7,8,9]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 1
AIC :  -31.775880587736808
BIC :  -31.266896935857467
FPE :  1.5847275488542232e-14
HQIC:  -31.56973847888264 

Lag Order = 2
AIC :  -32.59783102351524
BIC :  -31.66131249711438
FPE :  6.9697780007515665e-15
HQIC:  -32.21849745363904 

Lag Order = 3
AIC :  -33.0609124668401
BIC :  -31.69374443645674
FPE :  4.39234394475325e-15
HQIC:  -32.507092616322986 

Lag Order = 4
AIC :  -33.30368535771809
BIC :  -31.502714793796066
FPE :  3.4545689307644633e-15
HQIC:  -32.57406886595351 

Lag Order = 5
AIC :  -33.76999846619707
BIC :  -31.532033279133813
FPE :  2.1764554306257997e-15
HQIC:  -32.86325918017344 

Lag Order = 6
AIC :  -33.83988235988007
BIC :  -31.161690725588834
FPE :  2.0428283670117764e-15
HQIC:  -32.754678077764765 

Lag Order = 7
AIC :  -33.90428562090475
BIC :  -30.782595291504883
FPE :  1.933278645972626e-15
HQIC:  -32.63925783051571 

Lag Order = 8
AIC :  -34.15427949270424
BIC :  -30.585777091941775
FPE :  1.5248395394211516e-15
HQIC:  -32.70805310475275 

Lag